In [1]:
import os
from roboflow import Roboflow
from PIL import Image
import json
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np

In [2]:
current_dir = (os.getcwd())
current_dir

'd:\\DAFFA PINJEM BUAT BACKUP\\TA GASS\\FIX\\Yolo'

In [3]:
source_dir = './Multi-Object/'
target_dir = './Multi-Object-Result/'
predict_dir = './Predict/'

In [4]:
# Initialize API for coffee bean detection
rf = Roboflow(api_key="IluK8shPQ2JbRq6f5FVM")

# Reference : https://universe.roboflow.com/coffe-dataset/coffee-beans-znwfe/model/2
project = rf.workspace().project("coffee-beans-znwfe")
model = project.version(2).model

confident = 40
overlap = 30

loading Roboflow workspace...
loading Roboflow project...


In [5]:
def predict_image_multi(filename):
  # infer on a local image
  result = model.predict(f"{source_dir}{filename}", confidence=confident, overlap=overlap).json()

  # visualize your prediction
  model.predict(f"{source_dir}{filename}", confidence=confident, overlap=overlap).save(f"{target_dir}{filename}")
  return result

In [6]:
class_labels = ['Dark', 'Green', 'Light', 'Medium']

# define the path to the saved model
resnet50_path = 'resnet50-coffee-roast-level.pth'

# load the model
resnet50 = models.resnet50(pretrained=False)
resnet50.fc = nn.Linear(2048, 4)  # replace the last layer with a binary classifier
resnet50_state_dict = torch.load(resnet50_path, map_location=torch.device('cpu'))
resnet50.load_state_dict(resnet50_state_dict)

# resnet50 = torch.load(resnet50_path, map_location=torch.device('cpu'))
resnet50.eval()

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize( [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [8]:
file_images = os.listdir(source_dir)
result_list = []

for file_image in file_images:
    result = predict_image_multi(file_image)

    # Open the image file
    for index, predict in enumerate(result["predictions"]):
        img = Image.open(os.path.join(source_dir, file_image))

        # Set the crop region
        x, y, width, height = predict["x"], predict["y"], predict["width"], predict["height"]
        crop_region = (x-(int(width/2)), y-(int(height/2)), x+(int(width/2)), y+(int(height/2)))

        # Crop the image
        cropped_img = img.crop(crop_region)

        # Save cropped image
        cropped_img.save(f"./Predict/result-{index}.jpg")

        # Predict image
        image = transform(cropped_img).unsqueeze(0)

        # make a prediction
        with torch.no_grad():
            output = resnet50(image)

        # get the predicted class label
        predicted_class = np.argmax(output.numpy())
        predicted_label = class_labels[predicted_class.item()]

        # print the results
        print(f"Predicted class: {predicted_label}")

        # Display the original and cropped images
        # fig, (ax1, ax2) = plt.subplots(1, 2)
        # ax1.imshow(img)
        # ax1.set_title('Original Image')
        # ax2.imshow(cropped_img)
        # ax2.set_title('Cropped Image')
        # plt.show()
    result_list.append(result)

with open('result.json', 'w') as f:
    json.dump(result_list, f)

Predicted class: Dark
Predicted class: Medium
Predicted class: Dark
Predicted class: Dark
Predicted class: Dark
Predicted class: Green
Predicted class: Light
Predicted class: Green
Predicted class: Light
Predicted class: Green
Predicted class: Light


In [10]:
predict_img = os.listdir(predict_dir)

# loop through the images and make predictions
for image_path in predict_img:
    # load the image and apply the transformations
    image = Image.open(os.path.join(predict_dir, image_path))
    image = transform(image).unsqueeze(0)

    # make a prediction
    with torch.no_grad():
        output = resnet50(image)

    # get the predicted class label
    predicted_class = np.argmax(output.numpy())
    predicted_label = class_labels[predicted_class.item()]

    # print the results
    print(f"Image: {image_path} - Predicted class: {predicted_label}")

Image: result-0.jpg - Predicted class: Medium
Image: result-1.jpg - Predicted class: Medium
Image: result-10.jpg - Predicted class: Light
Image: result-2.jpg - Predicted class: Medium
Image: result-3.jpg - Predicted class: Medium
Image: result-4.jpg - Predicted class: Green
Image: result-5.jpg - Predicted class: Green
Image: result-6.jpg - Predicted class: Light
Image: result-7.jpg - Predicted class: Medium
Image: result-8.jpg - Predicted class: Green
Image: result-9.jpg - Predicted class: Green
